In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.io
from scipy.spatial.distance import cosine

import sys
sys.path.append('/Users/denisekittelmann/Documents/Python/BiMoL/code/util/')
from custom_pcn_dense import CustomDense, PredictiveCodingNetwork


In [ ]:

model = PredictiveCodingNetwork(vars=[1, 1, 1], beta=0.1)

# Load weights
dir_pcn = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/model_checkpoint_pcnoriginal_726_0.67.keras"
model.load_weights(dir_pcn)

# Save to a new file
updated_dir_pcn = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/updated_model_checkpoint.keras"
model.save(updated_dir_pcn)

/Users/denisekittelmann/miniconda3/envs/BiMo_3.9/lib/python3.9/site-packages/keras/src/saving/saving_api.py:107: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)


In [12]:
class CustomDense(tf.keras.layers.Dense):
    def call(self, inputs):
        """This works like a dense, except for the activation being called earlier."""
        # Apply the activation to the input first
        activated_input = self.activation(inputs)
        # Perform the matrix multiplication and add the bias
        output = tf.matmul(activated_input, self.kernel)
        if self.use_bias:
            output = output + self.bias
        return output


class PredictiveCodingNetwork(tf.keras.Sequential):
    def __init__(self, layers, vars, beta, **kwargs):
        """Initialize a PredictiveCodingNetwork"""
        super().__init__(layers, **kwargs)
        self.vars = tf.convert_to_tensor(vars, dtype=tf.float32)
        self.beta = beta

    def call_with_states(self, x):
        x_list = [x]
        for layer in self.layers:
            x = layer(x)
            x_list.append(x)
        return x_list

    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        # do the stuff we do in train_epochs
        outputs, errors = self.infer(x, y)
        self.update_params(outputs, errors)

        # Update metrics (includes the metric that tracks the loss)
        pred = self.call(x)
        for metric in self.metrics:
            metric.update_state(y, pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}
    
   
    def infer(self, x_batch, y_batch=None, n_iter=50, return_sequence=False):
        """Note: while model call, call with states and model evaluate take
        2D input, train_step and infer take stacked 3D inputs."""
        if return_sequence:
            errors_time = []
            states_time = []
        errors = [None for _ in range(len(self.layers))]
        f_x_arr = [None for _ in range(len(self.layers))]
        f_x_deriv_arr = [None for _ in range(len(self.layers))]
        shape = x_batch.shape
        batch_size = shape[0]

        for itr in range(n_iter):
            # if its the first itr, set x to the current forward call
            if itr == 0:
                x = self.call_with_states(x_batch)

                if y_batch is not None:
                  x[-1] = y_batch
            else:
                # update g and x only for consecutive iterations
                for l in range(1, len(self.layers)):
                    g = tf.multiply(tf.matmul(errors[l], self.layers[l].kernel, transpose_b=True), f_x_deriv_arr[l])
                    x[l] = x[l] + self.beta * (-errors[l-1] + g)

            # update f_x etc for every iteration
            for l in range(len(self.layers)):
                f_x = self.layers[l].activation(x[l])
                f_x_deriv_fn = self.get_activation_derivative(self.layers[l].activation)
                f_x_deriv = f_x_deriv_fn(x[l])
                f_x_arr[l] = f_x
                f_x_deriv_arr[l] = f_x_deriv
                errors[l] = (x[l + 1] - tf.matmul(f_x, self.layers[l].kernel) - self.layers[l].bias) / self.vars[l]
            
            if return_sequence:
                errors_time.append(errors)
                states_time.append(x)

        # return what we want to return
        if return_sequence:
            states_time = [tf.stack(tensors, axis=1) for tensors in zip(*states_time)]
            errors_time = [tf.stack(tensors, axis=1) for tensors in zip(*errors_time)]
            return states_time, errors_time
        else:
            return x, errors
    
    # We need to check if we actually need call here.
    # Now, call will give us the result of the network after the first inference step
    # If we want to have the results after the last inference step, we would need to change this
    #def call(self, inputs, training=False):
    #    """Call, but time distributed."""
    #    x, errors = self.infer(inputs, return_sequence=False)
    #    return x[-1]

    def update_params(self, x, errors):
        """Update the model parameters."""
        batch_size = tf.cast(tf.shape(x[0])[0], tf.float32)
        gradients = []
        for l, layer in enumerate(self.layers):
            grad_w = self.vars[-1] * (1 / batch_size) * tf.matmul(tf.transpose(self.layers[l].activation(x[l])), errors[l])
            grad_b = self.vars[-1] * (1 / batch_size) * tf.reduce_sum(errors[l], axis=0)
            gradients.append((-grad_w, layer.kernel))
            gradients.append((-grad_b, layer.bias))
        self.optimizer.apply_gradients(gradients)

    def get_activation_derivative(self, activation):
        """Return a function for the derivative of the given activation function."""
        activation_fn = tf.keras.activations.get(activation)
        if activation_fn == tf.keras.activations.linear:
            return lambda x: tf.ones_like(x)
        elif activation_fn == tf.keras.activations.tanh:
            return lambda x: 1 - tf.square(tf.nn.tanh(x))
        elif activation_fn == tf.keras.activations.sigmoid:
            return lambda x: tf.nn.sigmoid(x) * (1 - tf.nn.sigmoid(x))
        else:
            raise ValueError(f"{activation} not supported")
        

model = PredictiveCodingNetwork([CustomDense(units=6, activation="sigmoid"),
                                 CustomDense(units=4, activation="sigmoid"), 
                                 CustomDense(units=1, activation="sigmoid")], 
                                vars=[1, 1, 1], # variances. This is super useless and in the code only the last variance is used
                                beta=0.1)

In [9]:
dir_pcn = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/model_checkpoint_pcnoriginal_726_0.67.keras"
#test = tf.keras.models.load_model(dir_pcn, compile = False)

pcn = tf.keras.models.load_model(
    dir_pcn,
    custom_objects={
        "CustomDense": CustomDense,
        "PredictiveCodingNetwork": PredictiveCodingNetwork
    }
)

ValueError: Layers added to a Sequential model can only have a single positional argument, the input tensor. Layer InputLayer has multiple positional arguments: []

In [10]:
# Step 1: Load the model with compile=False to get the saved weights
screwed_up_model = tf.keras.models.load_model(dir_pcn, compile=False)

# Step 2: Create a new instance of the correct model class with the right parameters
# Substitute `param1` and `param2` with the actual parameters
#correct_model = MyCustomModel(param1=64, param2=10)  # example parameters

# Step 3: Manually load the weights from the incorrectly saved model into the new instance
#correct_model.set_weights(screwed_up_model.get_weights())

# Now `correct_model` is properly reconstructed with the correct parameters and weights


TypeError: Could not locate class 'PredictiveCodingNetwork'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'PredictiveCodingNetwork', 'config': {'name': 'predictive_coding_network', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 6214632496}, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 4704], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None}, {'module': None, 'class_name': 'CustomDense', 'config': {'name': 'custom_dense', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'units': 6, 'activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config

In [20]:
import tensorflow as tf
import zipfile
import os

# Step 1: Unzip the .keras file to access its internal structure
keras_file_path = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/model_checkpoint_pcnoriginal_726_0.67.keras"
unzip_path = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/pcn_tt/"

with zipfile.ZipFile(keras_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

# Step 2: Instantiate your custom model with the correct parameters
correct_model = model # Replace with your model parameters

# Step 3: Use a checkpoint to load weights from the 'variables' directory
#checkpoint = tf.train.Checkpoint(model=correct_model)
#checkpoint.restore(os.path.join(unzip_path)).assert_existing_objects_matched()

# Now `correct_model` has the loaded weights
correct_model.build([None, 4704])
correct_model.load_weights("/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/pcn_tt/model.weights.h5")

In [ ]:
import h5py
import numpy as np

# Path to the .h5 file containing weights
h5_file_path = "path_to_extracted_h5_file/variables.h5"

# Step 1: Open the .h5 file and inspect its structure
with h5py.File(h5_file_path, 'r') as h5_file:
    # Step 2: Extract weights as a list of numpy arrays
    # Here we look for all datasets under layer groups, which contain the weights
    weights_list = []
    for layer_name in h5_file.keys():
        layer_group = h5_file[layer_name]
        for weight_name in layer_group.keys():
            weight_dataset = layer_group[weight_name]
            weights_list.append(np.array(weight_dataset)

In [3]:
dir_pcn = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/updated_model_checkpoint.keras"

pcn = tf.keras.models.load_model(
    dir_pcn,
    custom_objects={
        "CustomDense": CustomDense,
        "PredictiveCodingNetwork": PredictiveCodingNetwork
    }
)

TypeError: <class 'custom_pcn_dense.PredictiveCodingNetwork'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'custom_pcn_dense', 'class_name': 'PredictiveCodingNetwork', 'config': {'name': 'predictive_coding_network', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 4509978288}, 'layers': [{'module': 'custom_pcn_dense', 'class_name': 'CustomDense', 'config': {'name': 'custom_dense', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'units': 6, 'activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': No